# self_intro_character_identification
Since we've explored the visual, audio, and subtitle tracks and extracted features from each, we can start to use them all together to accomplish broader *Moviegoer* goals. This notebook is the first example of this.

We'll be generating a list of possible characters, then looking for these names in self-identifications ("My name is Alice." or "I'm Ben."). Then we can build a composite, "average" encoding of their face, so we can track them throughout the film, every time we spot their face.

In [1]:
import sys
sys.path.append('../subtitle_features')
from subtitle_dataframes_io import *
from subtitle_auxiliary_io import *
sys.path.append('../vision_features')
from vision_dataframes_io import *
sys.path.append('../audio_features')
from audio_dataframes_io import *
from time_reference_io import *
import datetime

pd.set_option('display.max_colwidth', None)
nlp = spacy.load('en')

## Generating the Film's Character List
We'll generate the `subtitle_df` and `sentence_df` dataframes based off the subtitle file.

In [ ]:
subs = pysrt.open('../subtitles/plus_one.srt')
subtitle_df = generate_base_subtitle_df(subs)
subtitle_df = generate_subtitle_features(subtitle_df)
subtitle_df['cleaned_text'] = subtitle_df['concat_sep_text'].map(clean_line)
sentences = partition_sentences(remove_blanks(subtitle_df['cleaned_text'].tolist()), nlp)
subtitle_indices = tie_sentence_subtitle_indices(sentences, subtitle_df)
sentence_df = pd.DataFrame(list(zip(sentences, subtitle_indices)), columns=['sentence', 'subtitle_indices'])
sentence_df = generate_sentence_features(sentence_df, nlp)

We've previously defined two functions that will read through the subtitles and count up character names, either mentioned as dialogue, or labelling an offscreen speaker.

In [ ]:
chars_sub_mentions = character_subtitle_mentions(sentences, nlp)
chars_sub_mentions

In [ ]:
chars_offscreen_speakers = character_offscreen_speakers(subtitle_df)
chars_offscreen_speakers

We can take the most common names, and assume they're the main characters.

In [ ]:
characters = []

for character in chars_sub_mentions:
    if character[1] >= 10:
        characters.append(character[0].lower())

for character in chars_offscreen_speakers:
    if character[1] >= 5: 
        characters.append(character[0].lower())
        
characters = list(set(characters))
characters

## Finding Self-Introduction Sentences
For this exercise, we're going to identify characters solely based on self-introductions. We have a function to find phrases like "My name is Alice." or "I'm Ben."

In [ ]:
sentence_df[sentence_df['self_intro'].notnull()]

For now, we'll focus on Ben's self-introductions. The film has five sentences where he introduces himself. 

In [ ]:
ben_string = 'ben'

In [ ]:
sentence_df[sentence_df.self_intro.str.contains(ben_string, na=False, case=False)]

## Calculating the Film Times and Frames
From the `sentence_df`, we need to look for Ben's self-introductions and find the indices in `subtitle_df`, which contains times of the actual subtitles (not sentences).

In [ ]:
ben_indices = sentence_df[sentence_df.self_intro.str.contains(ben_string, na=False, case=False)].subtitle_indices.values
ben_indices

In [ ]:
ben_flattened_indices = np.concatenate(ben_indices).ravel()
ben_flattened_indices

We can see that one of these sentences spans two separate subtitles. We'll leave both of these in, because if it spans such a long duration, there's a good chance it'll have Ben's face onscreen.

In [ ]:
subtitle_df[subtitle_df.index.isin(ben_flattened_indices)]

For each subtitle, we'll calculate the `mid_time`, or the difference between the start and end times.

In [ ]:
mid_time = subtitle_mid_time(subtitle_df.iloc[506].start_time, subtitle_df.iloc[506].end_time)
time_to_frame(mid_time)

In [ ]:
mid_time_frames = []
for sub_index in ben_flattened_indices:
    mid_time = subtitle_mid_time(subtitle_df.iloc[sub_index].start_time, subtitle_df.iloc[sub_index].end_time)
    mid_time_frames.append(time_to_frame(mid_time))

mid_time_frames

## Collecting Face Encodings
We can now search all the frames (images) we gathered and collect the face encodings. From six frames, it looks like we were only able to find five encodings.

In [ ]:
movie_choice = 'plus_one'

ben_encodings = []

for frame_number in mid_time_frames:
    frame = load_frame(movie_choice, frame_number)

    locations = face_recognition.face_locations(frame, number_of_times_to_upsample=2)
    encodings = face_recognition.face_encodings(frame, locations)
    if encodings:
        ben_encodings.append(encodings)

In [ ]:
len(ben_encodings)

## Comparing Face Encodings
We can now use the `compare_faces()` function to compare the five encodings to each other. The first two faces appear to not match any other faces, while the final three all match each other.

In [ ]:
ben_scratch = ben_encodings.copy()
ben_flattened = []
for x in ben_scratch:
    for y in x:
        ben_flattened.append(y)
ben_compare = ben_flattened[0]
del ben_flattened[0]
face_recognition.compare_faces(ben_flattened, ben_compare)

In [ ]:
ben_scratch = ben_encodings.copy()
ben_flattened = []
for x in ben_scratch:
    for y in x:
        ben_flattened.append(y)
ben_compare = ben_flattened[1]
del ben_flattened[1]
face_recognition.compare_faces(ben_flattened, ben_compare)

In [ ]:
ben_scratch = ben_encodings.copy()
ben_flattened = []
for x in ben_scratch:
    for y in x:
        ben_flattened.append(y)
ben_compare = ben_flattened[2]
del ben_flattened[2]
face_recognition.compare_faces(ben_flattened, ben_compare)

In [ ]:
ben_scratch = ben_encodings.copy()
ben_flattened = []
for x in ben_scratch:
    for y in x:
        ben_flattened.append(y)
ben_compare = ben_flattened[3]
del ben_flattened[3]
face_recognition.compare_faces(ben_flattened, ben_compare)

In [ ]:
ben_scratch = ben_encodings.copy()
ben_flattened = []
for x in ben_scratch:
    for y in x:
        ben_flattened.append(y)
ben_compare = ben_flattened[4]
del ben_flattened[4]
face_recognition.compare_faces(ben_flattened, ben_compare)

In this example, we looked at five possible Ben face encodings, and found that three of them match each other. We can assume that this is Ben's face.

Finding a majority of faces that match each other is a good threshold. We can automate the above process to create an array of the three encodings that match one another.

In [ ]:
ben_scratch = ben_encodings.copy()
ben_flattened = []
for x in ben_scratch:
    for y in x:
        ben_flattened.append(y)

good_bens = []
x = 0
face_candidates = len(ben_flattened)

while x < face_candidates:
    ben_loop = ben_flattened.copy()
    ben_compare = ben_loop[x]
    del ben_loop[x]
    if sum(face_recognition.compare_faces(ben_loop, ben_compare)) >= (face_candidates - 1)/2:
        good_bens.append(ben_compare)
    x += 1

good_bens = np.array(good_bens)

len(good_bens)